<a href="https://colab.research.google.com/github/shouvikcirca/PCB_Defect_Detection/blob/master/LeNet5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeNet5.2

In [21]:
import torch
from torchvision import models, transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.metrics import confusion_matrix

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
# For Colab
# X = np.load('drive/My Drive/Copy of xtrain.npy')
# y = np.load('drive/My Drive/Copy of ytrain.npy')

In [4]:
# For local machine
X_train = pickle.load(open(f'xtrain298.pkl', 'rb'))
y_train = pickle.load(open(f'ytrain298.pkl', 'rb'))
X_test = pickle.load(open(f'xtest298.pkl', 'rb'))
y_test = pickle.load(open(f'ytest298.pkl', 'rb'))

In [5]:
# For Colab
# X = torch.from_numpy(X)
# X = X.permute(0,3,1,2)
# y = torch.from_numpy(y)
# X.shape, y.shape

Preprocessing Functions

In [6]:
def getNormalized(X,s):
    flattened_channels = X.reshape(3,-1)
    channel_mean = flattened_channels.mean(dim = 1)
    channel_stddev = flattened_channels.std(dim = 1)
    preprocess2 = transforms.Compose([
                      transforms.Normalize(channel_mean, channel_stddev)
    ])


    temptwo = torch.tensor([])
    for i in range(X.shape[0]):
        a = preprocess2(X[i])
        temptwo = torch.cat([temptwo, a.reshape(1,3,s,s)])
  
    return temptwo


def imageSetResize(newSize,X):
    preprocess1 = transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.Resize(newSize),
                        transforms.ToTensor()])
  
    temp = torch.tensor([])
    for i in range(X.shape[0]):
        a = preprocess1(X[i])
        temp = torch.cat([temp, a.reshape(1,3,newSize,newSize)])

    return temp 


def splitTrainTest(X,y):
    shuffled_indices = torch.randperm(X.shape[0])
    ul = math.floor(0.8*X.shape[0])
    train_indices = shuffled_indices[:ul]
    test_indices = shuffled_indices[ul:]
    # train_indices.shape[0] + test_indices.shape[0]
    X_train = X[train_indices]
    y_train = y[train_indices]  
    X_test = X[test_indices]
    y_test = y[test_indices]
    print('y_train -> [0]:{} [1]:{}'.format((y_train == 0).sum().item(), (y_train == 1).sum().item()))
    print('y_test -> [0]:{} [1]:{}'.format((y_test == 0).sum().item(), (y_test == 1).sum().item()))
    return X_train, y_train, X_test, y_test


def labelize(p):
    labelized_preds = []
    for i in p:
        l = 0. if i[0]>i[1] else 1.
        labelized_preds.append(l)

    return torch.tensor(labelized_preds)



def shuffle_and_batch(X,y,num,bs):
    shuffled_indices = torch.randperm(X.shape[0])
    newX = X[shuffled_indices]
    newY = y[shuffled_indices]

    X_batches = []
    y_batches = []
    for i in range(num):
        X_batches.append(X[i*bs:(i+1)*bs])
        y_batches.append(y_train[i*bs:(i+1)*bs])

    return X_batches, y_batches

Parameters

In [7]:
pars = []


#C1
c1 = nn.Conv2d(3,6, kernel_size = 5)
pars = pars + list(c1.parameters())

#S2
pool1 = nn.AvgPool2d(2, stride=2)

#C3
first_cons6_filterlist = []
second_cons6_filterlist = []
third_cons3_filterlist = []
fourth_last1_filterlist = []
for i in range(6):
    first_cons6_filterlist.append(nn.Conv2d(3,1,kernel_size = 5))
    pars = pars + list(first_cons6_filterlist[-1].parameters())
    
for i in range(6):
    second_cons6_filterlist.append(nn.Conv2d(4,1,kernel_size = 5))
    pars = pars + list(second_cons6_filterlist[-1].parameters())
    
for i in range(3):
    third_cons3_filterlist.append(nn.Conv2d(4,1,kernel_size = 5))
    pars = pars + list(third_cons3_filterlist[-1].parameters())

fourth_last1_filterlist = [nn.Conv2d(6,1,kernel_size = 5)]
pars = pars + list(fourth_last1_filterlist[-1].parameters())

#S4
pool2 = nn.AvgPool2d(2, stride=2)

#C5
conv3 = nn.Conv2d(16, 120, kernel_size = 5)
pars = pars + list(conv3.parameters())

#F6
ll1 = nn.Linear(120, 84)

#F7
ll2 = nn.Linear(84,2)
pars = pars + list(ll1.parameters()) + list(ll2.parameters())




In [8]:
criterion = nn.CrossEntropyLoss() # Loss Function

In [9]:
# def displayDetails(lone = None, ltwo=None):
#     if lone is not None:
#         a = lone
#         print('--------')
#         for i in a[0]:
#             print(i)
#         print('--------')
#         for i in a[1]:
#             print(i)
#         print('-------')
#         for i in a[2]:
#             print(i)
#         print('-------')
#         for i in a[3]:
#             print(i)
#     if ltwo is not None:
#         a = ltwo
#         for i in a:
#             print('-----')
#             for j in i:
#                 print(j.shape)

In [10]:
# displayDetails(lone = [first_cons6_filterlist,second_cons6_filterlist,third_cons3_filterlist,fourth_last1_filterlist])

Preprocessing

In [11]:
# resized_imageset = imageSetResize(32, X.float())
# normalized_imageset = getNormalized(resized_imageset.float(),32)

In [12]:
# X_train, y_train, X_test, y_test = splitTrainTest(resized_imageset, y)

In [13]:
# pickle.dump(X_train, open("xtrain298.pkl", 'wb'), protocol=4)
# pickle.dump(X_test, open("xtest298.pkl", 'wb'), protocol=4)
# pickle.dump(y_train, open("ytrain298.pkl", 'wb'), protocol=4)
# pickle.dump(y_test, open("ytest298.pkl", 'wb'), protocol=4)

In [14]:
def ef(li, c1_out):
    return torch.index_select(c1_out, 1, torch.tensor(li))

In [15]:
print('y_train -> [0]:{} [1]:{}'.format((y_train == 0).sum().item(), (y_train == 1).sum().item()))
print('y_test -> [0]:{} [1]:{}'.format((y_test == 0).sum().item(), (y_test == 1).sum().item()))

y_train -> [0]:119 [1]:119
y_test -> [0]:30 [1]:30


In [16]:
def feedForward(X):
    
    global c1
    global first_cons6_filterlist
    global second_cons6_filterlist
    global third_cons3_filterlist
    global fourth_last1_filterlist
    nos = X.shape[0]
    c1_out = c1(X)
    out = torch.tanh(pool1(c1_out))
    lione = [ef([0,1,2],out),ef([1,2,3],out), ef([2,3,4],out), ef([3,4,5],out), ef([0,4,5],out),ef([0,1,5],out)]
    litwo = [ef([0,1,2,3],out),ef([1,2,3,4],out), ef([2,3,4,5],out), ef([0,3,4,5],out), ef([0,1,4,5],out),ef([0,1,2,5],out)]
    lithree = [ef([0,1,3,4],out),ef([1,2,4,5],out), ef([0,2,3,5],out)]
    lifour = [ef([0,1,2,3,4,5],out)]
    feature_maps1 = []
    feature_maps2 = []
    feature_maps3 = []
    feature_maps4 = []
    for i in range(6):
        feature_maps1.append(first_cons6_filterlist[i](lione[i]))
    for i in range(6):
        feature_maps2.append(second_cons6_filterlist[i](litwo[i]))
    for i in range(3):
        feature_maps3.append(third_cons3_filterlist[i](lithree[i]))
    for i in range(1):
        feature_maps4.append(fourth_last1_filterlist[i](lifour[i]))
    fms = []
    fms.extend(feature_maps1)
    fms.extend(feature_maps2)
    fms.extend(feature_maps3)
    fms.extend(feature_maps4)
    tfms = torch.Tensor([])
    for i in fms:
        tfms = torch.cat([tfms, i], dim=1)
    c2_out = torch.tanh(pool2(tfms))
    c3_out = conv3(c2_out)
    c3_out = c3_out.reshape(nos,120)
    ll1_out = torch.tanh(ll1(c3_out))
    ll2_out = ll2(ll1_out)
    preds = nn.Softmax(dim=1)(ll2_out)
    
    return preds

In [17]:
optimizer = optim.Adam(pars,lr=1e-3) # Optimizer
# tb = SummaryWriter()

for epoch in range(10):
    print('Epoch {}'.format(epoch))
    X_batches, y_batches = shuffle_and_batch(X_train, y_train, 4, 67)
    for i in range(len(X_batches)):
        preds = feedForward(X_batches[i])
        loss = criterion(preds,y_batches[i].long())
        optimizer.zero_grad()
        loss.backward(retain_graph = True)
        optimizer.step()
    
    
    
    # Checking model on training set
    train_preds = feedForward(X_train)
    train_loss = criterion(train_preds, y_train.long())
    train_preds = labelize(train_preds)
    train_prediction_comparisons = (y_train == train_preds)
    train_accuracy = float(train_prediction_comparisons.sum())/float(y_train.shape[0])
    print('TrainLoss:{} TrainAccuracy:{}'.format(train_loss.item(), train_accuracy), end='  ')
  
    # Checking model on testing set
    test_preds = feedForward(X_test)
    test_loss = criterion(test_preds, y_test.long())
    test_preds = labelize(test_preds)
    test_prediction_comparisons = (y_test == test_preds)
    test_accuracy = float(test_prediction_comparisons.sum())/float(y_test.shape[0])
    print('TestLoss:{} TestAccuracy:{}'.format(test_loss.item(), test_accuracy))
    
#     tb.add_scalar('TrainLoss',train_loss, epoch)
#     tb.add_scalar('TestLoss',test_loss, epoch)
#     tb.add_scalar('TrainAccuracy', train_accuracy, epoch)
#     tb.add_scalar('TestAccuracy', test_accuracy, epoch)

Epoch 0
TrainLoss:0.6902676820755005 TrainAccuracy:0.5  TestLoss:0.6920586228370667 TestAccuracy:0.5
Epoch 1
TrainLoss:0.685481071472168 TrainAccuracy:0.6218487394957983  TestLoss:0.6897379159927368 TestAccuracy:0.6
Epoch 2
TrainLoss:0.6769767999649048 TrainAccuracy:0.6218487394957983  TestLoss:0.6865504384040833 TestAccuracy:0.5333333333333333
Epoch 3
TrainLoss:0.6602924466133118 TrainAccuracy:0.6176470588235294  TestLoss:0.6792013049125671 TestAccuracy:0.6
Epoch 4
TrainLoss:0.6393295526504517 TrainAccuracy:0.634453781512605  TestLoss:0.6724849939346313 TestAccuracy:0.5833333333333334
Epoch 5
TrainLoss:0.6189413666725159 TrainAccuracy:0.6470588235294118  TestLoss:0.6671918034553528 TestAccuracy:0.6
Epoch 6
TrainLoss:0.5985246896743774 TrainAccuracy:0.7016806722689075  TestLoss:0.6607180237770081 TestAccuracy:0.55
Epoch 7
TrainLoss:0.5782433748245239 TrainAccuracy:0.7226890756302521  TestLoss:0.6509877443313599 TestAccuracy:0.6166666666666667
Epoch 8
TrainLoss:0.563955545425415 TrainAc

Confusion Matrix

In [18]:
def cmatrix(target,preds):

    di = {}
    
    count00 = 0
    for i,j in zip(target,preds):
        if i==0.0 and j==0.0:
            count00+=1
    di['count00'] = count00
        
    
    count11 = 0
    for i,j in zip(target,preds):
        if i==1. and j==1.:
            count11+=1
    di['count11'] = count11

    
    count01 = 0
    for i,j in zip(target,preds):
        if i==0. and j==1.:
            count01+=1
    di['count01'] = count01
    
    count10 = 0
    for i,j in zip(target,preds):
        if i==1. and j==0.:
            count10+=1
    di['count10'] = count10
        
    print(di)

In [19]:
cmatrix(y_train, train_preds)

{'count00': 105, 'count11': 77, 'count01': 14, 'count10': 42}


In [20]:
confusion_matrix(y_train, train_preds)


array([[105,  14],
       [ 42,  77]])